In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import Counter
import time

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [2]:
start_time = time.time()

In [3]:
file_path_name = "../dataset/user_twitter_data.csv"

# Lire le fichier CSV
data = pd.read_csv(file_path_name)

# Extraire les colonnes des caractéristiques
features = data.columns[1:]  # Supposant que 'user_id' est la première colonne

# Extraire les données des caractéristiques
X = data[features]

# Calculer et sauvegarder la moyenne et le coefficient de variation de toutes les caractéristiques
mean_var_all = pd.DataFrame({
    'Mean': X.mean(),
    'CV': X.std() / (X.mean() +1e-10)
})
mean_var_all.to_csv('all_mean_cv_wo_standard.csv', index_label='Feature')

In [4]:
# Standardiser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def find_best_k(X_scaled, split_num, K_max=10):
    silhouette_scores = []
    k_range = range(2, K_max + 1)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X_scaled)
        labels = kmeans.labels_
        silhouette_scores.append(silhouette_score(X_scaled, labels))

    # Tracer le coefficient de silhouette
    plt.plot(k_range, silhouette_scores)
    plt.xlabel('Nombre de clusters')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score for K-means Clustering')
    plt.grid(True)

    # Sauvegarder le graphique dans un fichier
    plt.savefig('Silhouette_Score_K-means_Clustering' + '_cluste_' +str(split_num + 2) + '.png')
    plt.clf()
    best_k = k_range[np.argmax(silhouette_scores)]
    return best_k

In [ ]:
# Diviser les données en 9 parties
num_splits = 9
split_data = np.array_split(data, num_splits)

# Initialiser une liste vide pour stocker le nombre optimal de clusters pour chaque partie
best_k_list = []

# Traiter chaque partie des données
for i, split in enumerate(split_data):
    X_split = split[features]
    X_split_scaled = scaler.transform(X_split)
    best_k = find_best_k(X_split_scaled, i)
    best_k_list.append(best_k)
    print(f"Best number of clusters for split {i+1}: {best_k}")

# Compter le nombre d'apparitions de chaque nombre optimal de clusters
best_k_counts = Counter(best_k_list)
print("Best K counts:", best_k_counts)

# Choisir le nombre optimal de clusters le plus fréquent
final_best_k = best_k_counts.most_common(1)[0][0]
print(f"Final best number of clusters by majority vote: {final_best_k}")

# Stocker la moyenne et le CV de chaque cluster
cluster_stats = pd.DataFrame()

In [ ]:
# Définir une fonction pour effectuer le clustering K-means et trouver les outliers
def find_outliers(X_scaled, k):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X_scaled)
    distances = kmeans.transform(X_scaled)
    min_distances = np.min(distances, axis=1)
    threshold = np.mean(min_distances) + 2 * np.std(min_distances)
    outliers = min_distances > threshold
    return outliers
# Initialiser un DataFrame vide pour stocker tous les outliers
# all_outliers_data = pd.DataFrame()


# Effectuer le clustering K-means sur l'ensemble du dataset standardisé
kmeans = KMeans(n_clusters=final_best_k, random_state=0).fit(X_scaled)
labels = kmeans.labels_
data['Cluster'] = labels  # Assigner les labels des clusters aux données

# Initialiser un DataFrame vide pour collecter les statistiques des clusters
cluster_stats = pd.DataFrame()

# Calculer les statistiques des clusters pour chaque cluster
for cluster in range(final_best_k):
    cluster_data = data[data['Cluster'] == cluster][features]
    cluster_summary = pd.DataFrame({
        'Feature': features,
        f'Cluster {cluster+1} Mean': cluster_data.mean(),
        f'Cluster {cluster+1} CV': cluster_data.std() / (cluster_data.mean() +1e-10)
    })
    if cluster_stats.empty:
        cluster_stats = cluster_summary
    else:
        cluster_stats = pd.concat([cluster_stats, cluster_summary], axis=1)

# Identifier et rassembler les outliers si nécessaire
outliers = find_outliers(X_scaled, final_best_k)
outliers_data = data[outliers]
all_outliers_data = outliers_data  # Supposant que vous voulez stocker toutes les données des outliers

# Afficher toutes les informations des outliers
print("Final Outliers:")
print(all_outliers_data)

all_outliers_data.to_csv('all_outliers_data.csv', index=False, sep=',', lineterminator='\n')
print("Les données des outliers ont été sauvegardées dans all_outliers_data.csv")

# Afficher uniquement les IDs de tous les outliers
all_outliers_id = all_outliers_data["user_id"].values
print("Final Outliers IDs:")
print(all_outliers_id)
np.savetxt('all_outliers_id.csv', all_outliers_id, delimiter=',', header='Index,Value', comments='', fmt='%d')
print("Les IDs des outliers ont été sauvegardés dans all_outliers_id.csv")

cluster_stats.to_csv('clusters_mean_cv_wo_standard.csv', index=False)

end_time = time.time()
print("Total running time: {:.2f} seconds".format(end_time - start_time))